# Keto/Vegan Diet classifier
Argmax, a consulting firm specializing in search and recommendation solutions with offices in New York and Israel, is hiring entry-level Data Scientists and Machine Learning Engineers.

At Argmax, we prioritize strong coding skills and a proactive, “get-things-done” attitude over a perfect resume. As part of our selection process, candidates are required to complete a coding task demonstrating their practical abilities.

In this task, you’ll work with a large recipe dataset sourced from Allrecipes.com. Your challenge will be to classify recipes based on their ingredients, accurately identifying keto (low-carb) and vegan (no animal products) dishes.

Successfully completing this assignment is a crucial step toward joining Argmax’s talented team.

In [1]:
from opensearchpy import OpenSearch
from decouple import config
import pandas as pd

client = OpenSearch(
    hosts=[config('OPENSEARCH_URL', 'http://localhost:9200')],
    http_auth=None,
    use_ssl=False,
    verify_certs=False,
    ssl_show_warn=False,
)

# Recipes Index
Our data is stored in OpenSearch, and you can query it using either Elasticsearch syntax or SQL.
## Elasticsearch Syntax

In [2]:
query = {
    "query": {
        "match": {
            "description": { "query": "egg" }
        }
    }
}

res = client.search(
    index="recipes",
    body=query,
    size=2
)

hits = res['hits']['hits']
hits

[{'_index': 'recipes',
  '_id': 'eXVQeZcBIOqAXBOBXz16',
  '_score': 3.9817066,
  '_source': {'title': 'Genuine Egg Noodles',
   'description': 'These egg noodles are the original egg noodles.  ',
   'instructions': ['Combine flour, salt and baking powder. Mix in eggs and enough water to make the dough workable. Knead dough until stiff. Roll into ball and cut into quarters. Using 1/4 of the dough at a time, roll flat to about 1/8 inch use flour as needed, top and bottom, to prevent sticking. Peel up and roll from one end to the other. Cut roll into 3/8 inch strips. Noodles should be about 4 to 5 inches long depending on how thin it was originally flattened. Let dry for 1 to 3 hours.',
    'Cook like any pasta or, instead of drying first cook it fresh but make sure water is boiling and do not allow to stick. It takes practice to do this right.'],
   'ingredients': ['2 cups Durum wheat flour',
    '1/2 teaspoon salt',
    '1/4 teaspoon baking powder',
    '3 eggs',
    'water as needed'],

## SQL syntax

In [3]:
query = """
SELECT *
FROM recipes
LIMIT 100
OFFSET 100
"""

res = client.sql.query(body={'query': query})
df = pd.DataFrame(res["datarows"], columns=[c["name"] for c in res["schema"]])
df

,description,ingredients,instructions,photo_url,title
0,"Acidic ingredients, like the grapefruit and li...","[1/4 cup olive oil, 2 tablespoons grapefruit j...","[In a small bowl, combine the oil, grapefruit ...",http://images.media-allrecipes.com/global/reci...,Broiled Bluefish with Red Onion and Citrus Dre...
1,Roasting this stuffed chicken at 450 degrees F...,"[1/2 pound sliced white peasant bread, crust r...",[Preheat the oven to 450 degrees F. Spread the...,http://images.media-allrecipes.com/global/reci...,Roasted Chicken with Kale and Pancetta Stuffing
2,"A good-quality, at least slightly hot mustard ...","[2 tablespoons cooking oil, 1 onion, chopped, ...","[In a large saucepan, heat the oil over modera...",http://images.media-allrecipes.com/global/reci...,Lentils with Smoked Sausage and Carrots
3,The Good News: This dish combines omega-3-rich...,"[1/4 cup extra-virgin olive oil, plus more for...",[Preheat the oven to 400 degrees F. On a sheet...,http://images.media-allrecipes.com/global/reci...,Shrimp Boil with Three-Bean Salad
4,Red Thai curry paste gives this pureed carrot ...,"[1 cup jasmine or other long-grain rice, 1 1/2...",[Rinse the rice until the water runs clear. Pu...,http://images.media-allrecipes.com/global/reci...,Red Snapper on Rice with Red-Curry Carrot Sauce
...,...,...,...,...,...
95,.,"[3 tablespoons olive oil, 3/4 pound hot Italia...","[In a large deep frying pan or Dutch oven, hea...",http://images.media-allrecipes.com/global/reci...,Cavatelli with Sardinian Meat Sauce
96,.,"[3 tablespoons butter, 1 onion, chopped, 1 car...","[In a large pot, melt the butter over moderate...",http://images.media-allrecipes.com/global/reci...,Pumpkin Soup with Sage and Ham
97,.,"[1 quart fresh corn kernels, cut from the cob,...","[In a blender or food processor, combine the c...",http://images.media-allrecipes.com/global/reci...,Corn-and-Crabmeat Soup
98,Lemon zest and rosemary placed in the cavity o...,"[1 (3 pound) chicken, 1 tablespoon dried rosem...",[Heat the oven to 425 degrees F. Rub the cavit...,http://images.media-allrecipes.com/global/reci...,Roast Chicken with Rosemary and Lemon


# Task Instructions

Your goal is to implement two classifiers:

1.	Vegan Meal Classifier
1.	Keto Meal Classifier

Unlike typical supervised machine learning tasks, the labels are not provided in the dataset. Instead, you will rely on clear and verifiable definitions to classify each meal based on its ingredients.

### Definitions:

1. **Vegan Meal**: Contains no animal products whatsoever (no eggs, milk, meat, etc.).
1. **Keto Meal**: Contains no ingredients with more than 10g of carbohydrates per 100g serving. For example, eggs are keto-friendly, while apples are not.

Note that some meals may meet both vegan and keto criteria (e.g., meals containing avocados), though most meals typically fall into neither category.

## Example heuristic:

### Limitations of the Simplistic Heuristic

The heuristic described above is straightforward but can lead to numerous false positives and negatives due to its reliance on keyword matching. Common examples of incorrect classifications include:
- "Peanut butter" being misclassified as non-vegan, as “butter” is incorrectly assumed to imply dairy.
- "eggless" recipes being misclassified as non-vegan, due to the substring “egg.”
- Animal-derived ingredients such as “pork” and “bacon” being incorrectly identified as vegan, as they may not be explicitly listed in the keyword set.


# Submission
## 1. Implement Diet Classifiers
Complete the two classifier functions in the diet_classifiers.py file within this repository. Ensure your implementation correctly identifies “keto” and “vegan” meals. After implementing these functions, verify that the Flask server displays the appropriate badges (“keto” and “vegan”) next to the corresponding recipes.

> **Note**
>
> This repo contains two `diet_classifiers.py` files:
> 1. One in this folder (`nb/src/diet_classifiers.py`)
> 2. One in the Flask web app folder (`web/src/diet_classifiers.py`)
>
> You can develop your solution here in the notebook environment, but to apply your solution 
> to the Flask app you will need to copy your implementation into the `diet_classifiers.py` 
> file in the Flask folder!!!

For your convenience, you can sanity check your solution on a subset of labeled recipes by running `diet_classifiers.py`

In [4]:
! python diet_classifiers.py --ground_truth /usr/src/data/ground_truth_sample.csv

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
===Keto===
              precision    recall  f1-score   support

       False       0.84      0.90      0.87        60
        True       0.83      0.75      0.79        40

    accuracy                           0.84       100
   macro avg       0.84      0.82      0.83       100
weighted avg       0.84      0.84      0.84       100

===Vegan===
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/me

## 2. Repository Setup
Create a **private** GitHub repository for your solution, and invite the GitHub user `argmax2025` as a collaborator. **Do not** share your implementation using a **forked** repository.

## 3. Application Form
Once you’ve completed the implementation and shared your private GitHub repository with argmax2025, please fill out the appropriate application form:
1. [US Application Form](https://forms.clickup.com/25655193/f/rexwt-1832/L0YE9OKG2FQIC3AYRR)
2.  [IL Application Form](https://forms.clickup.com/25655193/f/rexwt-1812/IP26WXR9X4P6I4LGQ6)


Your application will not be considered complete until this form is submitted.

## Evaluation process


Your submission will be assessed based on the following criteria:


1.	**Readability & Logic** – Clearly explain your approach, including your reasoning and any assumptions. If you relied on external resources (e.g., ingredient databases, nutrition datasets), be sure to cite them.
2.	**Executability** – Your code should run as is when cloned from your GitHub repository. Ensure that all paths are relative, syntax is correct, and no manual setup is required.
3.	**Accuracy** – Your classifiers will be evaluated against a holdout set of 20,000 recipes with verified labels. Performance will be compared to the ground truth.
data.


## Next steps
If your submission passes the initial review, you’ll be invited to a 3-hour live coding interview, where you’ll be asked to extend and adapt your solution in real time.

Please make sure you join from a quiet environment and have access to a Python-ready workstation capable of running your submitted project.